## Flux simulation

In [1]:
import cobra

# Open the metabolic models
s12 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')
kt2440 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')

In [2]:
# The fluxes from 13C-based metabolic flux analysis of S12
r_id = ['GND','PGI','EDA','TKT2','FBA','PGK','ENO','PYK','PDH','PPCK','ME2','MDH','FUM','PC','ICDHyr','ASPTA']
exp = [0.6, -0.4, 7, 0.1, -1.1, -4.7, 3.7, 4.4, 6.9, 1.1, 2.6, 5.1, 4.8, 5.4, 5.6]

for r in r_id:
    print(s12.reactions.get_by_id(r))

GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c
PGI: g6p_c <=> f6p_c
EDA: 2ddg6p_c --> g3p_c + pyr_c
TKT2: e4p_c + xu5p__D_c <=> f6p_c + g3p_c
FBA: fdp_c <=> dhap_c + g3p_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
ENO: 2pg_c <=> h2o_c + pep_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c
PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
PPCK: atp_c + oaa_c --> adp_c + co2_c + pep_c
ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
FUM: fum_c + h2o_c <=> mal__L_c
PC: atp_c + hco3_c + pyr_c --> adp_c + h_c + oaa_c + pi_c
ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
ASPTA: akg_c + asp__L_c <=> glu__L_c + oaa_c


In [3]:
# set the glucose uptake rate
s12.reactions.get_by_id('EX_glc__D_e').lower_bound = -6.5
kt2440.reactions.get_by_id('EX_glc__D_e').lower_bound = -6.5

In [4]:
# To generate Figures 4C and S3 in the reference paper

import pandas as pd
import numpy as np

s12_c13 = []
kt2440_c13 = []

for r in r_id:
    if r == 'ICDHyr':
        ICDHyr=kt2440.optimize().fluxes[r]
    elif r == 'ASPTA':
        ASPTA=kt2440.optimize().fluxes[r]
    else:
        kt2440_c13.append(kt2440.optimize().fluxes[r])
kt2440_c13.append(ICDHyr-ASPTA)

for r in r_id:
    if r == 'ICDHyr':
        ICDHyr=s12.optimize().fluxes[r]
    elif r == 'ASPTA':
        ASPTA=s12.optimize().fluxes[r]
    else:
        s12_c13.append(s12.optimize().fluxes[r])
s12_c13.append(ICDHyr-ASPTA)

index = ['GND','PGI','EDA','TKT2','FBA','PGK','ENO','PYK','PDH','PPCK','ME2','MDH','FUM','PC','ICDHyr_ASPTA']
df = pd.DataFrame({'experimental':exp,'iSH1474_c13':s12_c13,'iJN1462_c13':kt2440_c13},index=index)

display(df)
print('iSH1474 pearson coefficient (r):',np.corrcoef(s12_c13,exp)[0,1])
print('iJN1462 pearson coefficient (r):',np.corrcoef(kt2440_c13,exp)[0,1])

,experimental,iSH1474_c13,iJN1462_c13
GND,0.6,0.891788,0.876962
PGI,-0.4,-0.022558,-0.022099
EDA,7.0,5.608212,5.623038
TKT2,0.1,-0.092853,-0.092745
FBA,-1.1,0.000000,0.000000
PGK,-4.7,-5.470036,-5.486338
ENO,3.7,4.511763,4.302286
PYK,4.4,3.964083,3.489437
PDH,6.9,7.621426,0.000000
PPCK,1.1,0.000000,0.000000


iSH1474 pearson coefficient (r): 0.8756347133703917
iJN1462 pearson coefficient (r): 0.7630067531672781
